<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH9_13_Doc2Vec%EC%9C%BC%EB%A1%9C_%EA%B3%B5%EC%8B%9C_%EC%82%AC%EC%97%85%EB%B3%B4%EA%B3%A0%EC%84%9C_%EC%9C%A0%EC%82%AC%EB%8F%84_%EA%B3%84%EC%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Word2Vec은 단어를 임베딩하는 워드 임베딩 알고리즘임. Doc2Vec은 문서의 임베딩을 얻을 수 있도록 만들어진 알고리즘임

* 논문 제목 : Distributed Representations of Sentences and Documents
* 논문 링크 : https://arxiv.org/abs/1405.4053

여기서는 전자공시스템의 사업보고서를 Doc2Vec을 통해 학습하고 특정회사와 사업보고서가 유사한 회사를 찾는 모델 실습


### 1. 공시사업 보고서 로드 및 전처리
-------------------
Mecab 먼저 설치!!

In [1]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 4.08 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 42.8 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-03-18 11:28:22--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c5:2ef4, 2406:da00:ff00::

In [2]:
# dart.csv 파일 다운로드
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL" -O dart.csv && rm -rf /tmp/cookies.txt

--2023-03-18 11:31:22--  https://docs.google.com/uc?export=download&confirm=t&id=1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL
Resolving docs.google.com (docs.google.com)... 74.125.137.102, 74.125.137.101, 74.125.137.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.137.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-7k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/dlb2th0a56t3qjfarg8v22al5nrikd50/1679139075000/17609157229046208934/*/1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL?e=download&uuid=ce81fb82-f4d5-4870-bbcd-70e2ebe8ed20 [following]
--2023-03-18 11:31:22--  https://doc-04-7k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/dlb2th0a56t3qjfarg8v22al5nrikd50/1679139075000/17609157229046208934/*/1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL?e=download&uuid=ce81fb82-f4d5-4870-bbcd-70e2ebe8ed20
Resolving doc-04-7k-docs.googleusercontent.com (doc-04-7k-docs.googleusercontent.com)... 142.251.2.132, 2

In [3]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

In [4]:
## 데이터 로드 
data = pd.read_csv('dart.csv', sep=',')
data.tail()

,code,market,name,business
2584,950160,KOSDAQ,코오롱티슈진,II. 사업의 내용\n당사는 1999년 6월 9일 세포 유전자 기술 Platform...
2585,950170,KOSDAQ,JTC,II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n(1) 산업의 특성\n...
2586,950180,KOSDAQ,SNK,"II. 사업의 내용\n1. 사업의 개요\n당사는 연결실체 기준으로 모바일, 콘솔, ..."
2587,950190,KOSDAQ,미투젠,NaN
2588,950200,KOSDAQ,소마젠(Reg.S),NaN


In [5]:
data = data.dropna()  # 결측값 제거거
data.tail()

,code,market,name,business
2582,950130,KOSDAQ,엑세스바이오,II. 사업의 내용\n1. 사업의 개\n요\n당사는 체외진단 기술을 토대로 말라리아...
2583,950140,KOSDAQ,잉글우드랩,II. 사업의 내용\n1. 사업의 개요\n가. 화장품 ODM산업 개요\nODM은 O...
2584,950160,KOSDAQ,코오롱티슈진,II. 사업의 내용\n당사는 1999년 6월 9일 세포 유전자 기술 Platform...
2585,950170,KOSDAQ,JTC,II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n(1) 산업의 특성\n...
2586,950180,KOSDAQ,SNK,"II. 사업의 내용\n1. 사업의 개요\n당사는 연결실체 기준으로 모바일, 콘솔, ..."


In [6]:
data.shape

(2295, 4)

총 2295개로 정보로 구성. 여기에서 사용할 정보는 business 컬럼이기 때문에, 이에 대해 형태소 분석 수행


이와 동시에, Doc2Vec 학습을 위해 필요한 입력형태로 데이터 변환 필요. Doc2Vec 학습을 위해 '제목'과 '토큰화된 본문' 두가지가 필요함

TaggedDocument의 tags에 해당 문서의 '제목', words에 '본문'을 넣은 형태로 결과를 저장해야 됨 ==> tagged_corpus_list

In [7]:
mecab = Mecab()
tagged_corpus_list = []

for index, row in tqdm(data.iterrows(), total=(len(data))):
  text = row['business']
  tag = row['name']
  tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))  # tag를 반드시 리스트 형식으로!!

100%|██████████| 2295/2295 [02:36<00:00, 14.68it/s]


In [8]:
len(tagged_corpus_list)

2295

In [9]:
# 전처리 결과 확인
tagged_corpus_list[0]

TaggedDocument(words=['II', '.', '사업', '의', '내용', '1', '.', '사업', '의', '개요', '가', '.', '일반', '적', '인', '사항', '기업', '회계', '기준', '서', '제', '1110', '호', '"', '연결', '재무제표', '"', '의', '의하', '여', '2018', '년', '12', '월', '17', '일', '에', '설립', '한', '동화', '크립톤', '기업가', '정신', '제일', '호', '창업', '벤처', '전문', '사모', '투자', '합자회사', '를', '종속', '회사', '에', '편입', '하', '였', '습니다', '.', '나', '.', '지배', '기업', '의', '현황', '1', ')', '산업', '의', '특성', '제약', '산업', '은', '약학', ',', '화학', ',', '생물학', '등', '여러', '분야', '의', '지식', '과', '기술', '이', '복합', '적', '으로', '필요', '한', '기술', '·', '지식', '집약', '형', '산업', '으로', '인간', '의', '생명', '과', '보건', '에', '직접', '적', '으로', '관련', '된', '제품', '을', '생산', '하', '고', ',', '개발', '·', '제조', '·', '유통', '·', '가격', '등', '을', '정부', '에서', '엄격', '하', '게', '규제', '하', '고', '관리', '하', '는', '산업', '적', '특성', '을', '가지', '고', '있', '습니다', '.', '또한', '신약', '개발', '에', '는', '막대', '한', '시간', '과', '비용', '이', '소요', '되', '며', ',', '성공', '확률', '또한', '높', '지', '않', '으나', '신약', '개발', '에', '성공', '할', '경우', '엔', '엄청난

### 2. Doc2Vec 학습 및 테스트
-------------------------
토큰화 외에 불용어 제거 등 별도의 추가 전처리를 하지 않은 채 모델 개발

In [10]:
from gensim.models import doc2vec

In [11]:
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# vocabulary 빌드
model.build_vocab(tagged_corpus_list)
print(f"Tag size: {len(model.docvecs.doctags.keys())}", end=' / ')

Tag size: 2295 / 

In [12]:
model.corpus_count

2295

In [13]:
# 모델 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('dart.doc2vec')

다음의 3개의 파일이 생성됨
* dart.doc2vec
* dart.doc2vec.trainables.syn1neg.npy
* dart.doc2vec.wv.vectors.npy

테스트 진행


In [14]:
similar_doc = model.docvecs.most_similar('동화약품')

In [15]:
similar_doc

[('종근당', 0.5710541009902954),
 ('일양약품', 0.5386483073234558),
 ('제일약품', 0.5288128852844238),
 ('유한양행', 0.5121499300003052),
 ('영진약품', 0.5110045671463013),
 ('우리들제약', 0.505203127861023),
 ('삼일제약', 0.4913202226161957),
 ('현대약품', 0.48686501383781433),
 ('동아에스티', 0.4850732088088989),
 ('대원제약', 0.4807552695274353)]

In [16]:
model.docvecs.most_similar('하이트진로')

[('하이트진로홀딩스', 0.8595354557037354),
 ('보해양조', 0.503746747970581),
 ('무학', 0.4944235384464264),
 ('국순당', 0.4765230417251587),
 ('롯데칠성', 0.46050411462783813),
 ('두산퓨얼셀', 0.4004940688610077),
 ('금비', 0.39672207832336426),
 ('대한제당', 0.3905535936355591),
 ('삼양홀딩스', 0.38997822999954224),
 ('서부T&D', 0.38894760608673096)]

In [18]:
model.docvecs.most_similar('LG이노텍')

[('LG전자', 0.5546497106552124),
 ('LG', 0.49773839116096497),
 ('삼성전기', 0.4811190366744995),
 ('LG디스플레이', 0.47333240509033203),
 ('서울바이오시스', 0.44757139682769775),
 ('서울반도체', 0.4427380859851837),
 ('KEC', 0.4393792748451233),
 ('코렌', 0.40027081966400146),
 ('코아시아', 0.39966708421707153),
 ('캠시스', 0.39958277344703674)]

In [19]:
model.docvecs.most_similar('메리츠화재')

[('메리츠금융지주', 0.7202873229980469),
 ('DB손해보험', 0.6723909974098206),
 ('한화손해보험', 0.6719530820846558),
 ('한화생명', 0.6570014953613281),
 ('롯데손해보험', 0.6436102390289307),
 ('흥국화재', 0.642727255821228),
 ('현대해상', 0.629374086856842),
 ('삼성화재', 0.5768622159957886),
 ('동양생명', 0.569711446762085),
 ('코리안리', 0.5679256319999695)]

In [20]:
model.docvecs.most_similar('카카오')

[('NAVER', 0.4985593557357788),
 ('네오위즈', 0.49822086095809937),
 ('카카오게임즈', 0.4957650303840637),
 ('네오위즈홀딩스', 0.4815990924835205),
 ('퓨쳐스트림네트웍스', 0.4650757312774658),
 ('넷마블', 0.46401673555374146),
 ('엠게임', 0.4592636823654175),
 ('케어랩스', 0.45335352420806885),
 ('액션스퀘어', 0.4513722062110901),
 ('컴투스', 0.44941404461860657)]

### 3. document clustering
---------------------
KMeans++를 이용하여 유사도로 클러스터링 분석

KMeans++는 초기 중심위치를 설정하기 위한 알고리즘임. 

1. 중심위치를 저장할 집합 M 준비
2. 하나의 중심위치 $μ_0$ 를 랜덤하게 선택하여 M에 넣는다.
3. M에 속하지 않는 모든 표본 $x_i$에 대해 거리 $d(M,x_i)$를 계산. $d(M,x_i)$는 M 안의 모든 샘플 $μ_k$에 대해 $d(μ_k,x_i)$를 계산하여 가장 작은 값 선택
4. $d(M,x_i)$에 비례한 확률로 다음 중심위치 $μ$를 선택.
5. K개의 중심위치를 선택할 때까지 반복
6. K-평균 방법 사용

In [21]:
from sklearn.cluster import KMeans

In [48]:
cluster_dict = {i:[] for i in range(0, 10)}
cluster_dict_title = {i:[] for i in range(0, 10)}

In [51]:
clustering_method = KMeans(n_clusters=10, n_init=1, init='k-means++', random_state=0)

In [52]:
X = model.docvecs.vectors_docs  # text 전체 벡터 
clustering_method.fit(X)

KMeans(n_clusters=10, n_init=1, random_state=0)

In [53]:
for text_tag, label in zip(tagged_corpus_list, clustering_method.labels_):
  text, tag = text_tag
  cluster_dict[label].append(text)
  cluster_dict_title[label].append(tag)

In [54]:
for label, lst in cluster_dict_title.items():
  print(f"cluster {label}")
  for x in lst:
    print(x)
  print("--"*30)

cluster 0
['CJ대한통운']
['성창기업지주']
['대림산업']
['현대건설']
['남광토건']
['삼부토건']
['동양']
['이화공영']
['대림건설']
['도화엔지니어링']
['삼일기업공사']
['범양건영']
['화성산업']
['진흥기업']
['선창산업']
['금호산업']
['코오롱글로벌']
['한일홀딩스']
['쌍용양회']
['한진중공업홀딩스']
['삼일씨엔에스']
['삼천리']
['한신공영']
['성신양회']
['POSCO']
['동부건설']
['국영지앤엠']
['GS건설']
['한일현대시멘트']
['보성파워텍']
['모헨즈']
['일신석재']
['벽산']
['대원']
['이건산업']
['NI스틸']
['부스타']
['삼성중공업']
['아이에스동서']
['삼호개발']
['서한']
['부산산업']
['세보엠이씨']
['경동인베스트']
['HDC']
['동원개발']
['일성건설']
['계룡건설']
['까뮤이앤씨']
['금강공업']
['한라']
['부산가스']
['예스코홀딩스']
['한국전력']
['이테크건설']
['신원종합개발']
['서울가스']
['세명전기']
['현대엘리베이']
['삼목에스폼']
['일진머티리얼즈']
['시공테크']
['KCC건설']
['정산애강']
['한국종합기술']
['유진기업']
['한솔홈데코']
['동신건설']
['아난티']
['특수건설']
['삼성엔지니어링']
['동아지질']
['삼성물산']
['제룡전기']
['파라텍']
['두산중공업']
['신세계건설']
['인천도시가스']
['해성산업']
['서희건설']
['금화피에스시']
['한국가스공사']
['성도이엔지']
['희림']
['제일테크노스']
['삼표시멘트']
['이건홀딩스']
['HDC아이콘트롤스']
['젬백스지오']
['상지카일룸']
['에이치엘비파워']
['자연과환경']
['우원개발']
['대우건설']
['한전KPS']
['한전기술']
['지에스이']
['한국테크놀로지']
['한미글로벌']
['유신']
['다스코']
['인선이엔티']
['뉴보텍']
['KT서브

In [55]:
for i in range(10):
  print(f"label {i} size: {len(cluster_dict_title[i])}")

label 0 size: 153
label 1 size: 108
label 2 size: 201
label 3 size: 292
label 4 size: 265
label 5 size: 176
label 6 size: 313
label 7 size: 230
label 8 size: 223
label 9 size: 334


In [56]:
cluster_dict_title[6]  # label6는 IT 기업으로 이루어짐

[['삼양홀딩스'],
 ['SK하이닉스'],
 ['영풍'],
 ['DB하이텍'],
 ['삼화콘덴서'],
 ['미원상사'],
 ['디아이'],
 ['삼영화학'],
 ['롯데정밀화학'],
 ['삼익THK'],
 ['송원산업'],
 ['한솔테크닉스'],
 ['덕성'],
 ['코스모신소재'],
 ['동진쎄미켐'],
 ['코스모화학'],
 ['삼영전자'],
 ['삼성전자'],
 ['삼성SDI'],
 ['이수페타시스'],
 ['코리아써키트'],
 ['대덕'],
 ['대동전자'],
 ['율촌화학'],
 ['삼성전기'],
 ['참엔지니어링'],
 ['OCI'],
 ['우진아이엔에스'],
 ['LG이노텍'],
 ['삼화전자'],
 ['한농화성'],
 ['SKC'],
 ['신성이엔지'],
 ['뉴인텍'],
 ['경인양행'],
 ['세우글로벌'],
 ['원익큐브'],
 ['한솔케미칼'],
 ['동원시스템즈'],
 ['성문전자'],
 ['에이엔피'],
 ['광전자'],
 ['일진디스플'],
 ['디씨엠'],
 ['피에스엠씨'],
 ['대원화성'],
 ['시노펙스'],
 ['한국단자'],
 ['미래산업'],
 ['동화기업'],
 ['상보'],
 ['케이씨'],
 ['원익홀딩스'],
 ['에스에이엠티'],
 ['피에스케이홀딩스'],
 ['동일기연'],
 ['엠케이전자'],
 ['시그네틱스'],
 ['자화전자'],
 ['코웰패션'],
 ['네패스'],
 ['LG디스플레이'],
 ['아비코전자'],
 ['위지트'],
 ['유니셈'],
 ['SK머티리얼즈'],
 ['SFA반도체'],
 ['심텍홀딩스'],
 ['에프에스티'],
 ['솔브레인홀딩스'],
 ['우리조명'],
 ['루멘스'],
 ['에이아이비트'],
 ['에스티아이'],
 ['아이씨디'],
 ['이라이콤'],
 ['케이피엠테크'],
 ['에스씨디'],
 ['새로닉스'],
 ['한미반도체'],
 ['성호전자'],
 ['케이맥'],
 ['크로바하이텍'],
 ['에이치엔에스하이텍'],
 ['한양이엔지'],
 ['성우테크론'],
 ['서